Analysis of transaction visibility in the public mempool before block inclusion on Ethereum mainnet.

**Methodology:** A transaction is counted as "seen in mempool" only if it was observed by our sentries *before* the slot start time of the block that included it. This corrects for transactions that appear in the mempool after block propagation.

In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from loaders import load_parquet, display_sql

# Transaction type labels and colors
TX_TYPE_LABELS = {
    0: "Legacy",
    1: "Access list",
    2: "EIP-1559",
    3: "Blob",
    4: "EIP-7702",
}

TX_TYPE_COLORS = {
    0: "#636EFA",
    1: "#EF553B",
    2: "#00CC96",
    3: "#AB63FA",
    4: "#FFA15A",
}

# Histogram bucket labels (log2 seconds, up to 1 hour)
HIST_LABELS = [
    "<0.5s", "0.5-1s", "1-2s", "2-4s", "4-8s", "8-16s",
    "16-32s", "32s-1m", "1-2m", "2-4m", "4-8m", "8-17m",
    "17-34m", "34-60m", ">=1h"
]

target_date = None  # Set via papermill, or auto-detect from manifest

In [ ]:
display_sql("mempool_availability", target_date)

In [ ]:
df = pl.from_pandas(load_parquet("mempool_availability", target_date))

df = df.with_columns(
    pl.col("tx_type").replace_strict(TX_TYPE_LABELS, default=None).alias("tx_type_label"),
    (pl.col("seen_before_slot") / pl.col("total_txs") * 100).alias("coverage_pct"),
    (pl.col("total_txs") - pl.col("seen_before_slot") - pl.col("seen_after_slot")).alias("never_seen"),
)

# Extract p50 age from percentiles array (index 0)
df = df.with_columns(
    pl.col("age_percentiles_ms").list.get(0).alias("p50_age_ms"),
)
df = df.with_columns(
    (pl.col("p50_age_ms") / 1000).alias("p50_age_s"),
)

# Add hour column for time-series aggregation
df = df.with_columns(
    pl.col("slot_start_date_time").dt.truncate("1h").alias("hour"),
)

total = df["total_txs"].sum()
before = df["seen_before_slot"].sum()
after = df["seen_after_slot"].sum()
never = total - before - after

print(f"Loaded {len(df):,} slot/type rows")
print(f"Slots: {df['slot'].n_unique():,}")
print(f"Total transactions: {total:,}")
print(f"  Seen before slot: {before:,} ({100*before/total:.1f}%)")
print(f"  Seen after slot:  {after:,} ({100*after/total:.1f}%)")
print(f"  Never seen:       {never:,} ({100*never/total:.1f}%)")

## Coverage by transaction type

Percentage of transactions seen in the public mempool *before* the slot they were included in. Low coverage indicates private or MEV transactions that bypass the public mempool or are submitted just-in-time.

In [ ]:
# Aggregate by type
df_summary = df.group_by(["tx_type", "tx_type_label"]).agg(
    pl.col("total_txs").sum(),
    pl.col("seen_before_slot").sum(),
    pl.col("seen_after_slot").sum(),
)
df_summary = df_summary.with_columns(
    (pl.col("total_txs") - pl.col("seen_before_slot") - pl.col("seen_after_slot")).alias("never_seen"),
    (pl.col("seen_before_slot") / pl.col("total_txs") * 100).alias("before_pct"),
    (pl.col("seen_after_slot") / pl.col("total_txs") * 100).alias("after_pct"),
)
df_summary = df_summary.with_columns(
    (pl.col("never_seen") / pl.col("total_txs") * 100).alias("never_pct"),
)

# Display summary table
summary_display = df_summary.select(
    pl.col("tx_type_label").alias("Type"),
    pl.col("total_txs").alias("Total"),
    pl.col("before_pct").round(1).alias("Before slot %"),
    pl.col("after_pct").round(1).alias("After slot %"),
    pl.col("never_pct").round(1).alias("Never seen %"),
)
summary_display.to_pandas()

In [ ]:
# Coverage stacked bar chart showing before/after/never breakdown
df_summary_pd = df_summary.to_pandas()

fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_summary_pd["tx_type_label"],
    y=df_summary_pd["before_pct"],
    name="Before slot (public)",
    marker_color="#27ae60",
    text=df_summary_pd["before_pct"].round(1),
    textposition="inside",
))
fig.add_trace(go.Bar(
    x=df_summary_pd["tx_type_label"],
    y=df_summary_pd["after_pct"],
    name="After slot (propagated)",
    marker_color="#3498db",
    text=df_summary_pd["after_pct"].round(1),
    textposition="inside",
))
fig.add_trace(go.Bar(
    x=df_summary_pd["tx_type_label"],
    y=df_summary_pd["never_pct"],
    name="Never seen (private)",
    marker_color="#95a5a6",
    text=df_summary_pd["never_pct"].round(1),
    textposition="inside",
))

fig.update_traces(texttemplate="%{text:.1f}%")
fig.update_layout(
    barmode="stack",
    margin=dict(l=60, r=30, t=30, b=60),
    xaxis=dict(title="Transaction type"),
    yaxis=dict(title="Percentage", range=[0, 105]),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
    height=400,
)
fig.show(config={"responsive": True})

## Hourly coverage trends

Mempool visibility percentage over time for each transaction type. Blob transactions (type 3) typically have the highest visibility since they propagate through the public network.

In [ ]:
# Aggregate to hourly for time-series
df_hourly = df.group_by(["hour", "tx_type", "tx_type_label"]).agg(
    pl.col("total_txs").sum(),
    pl.col("seen_before_slot").sum(),
    pl.col("seen_after_slot").sum(),
)
df_hourly = df_hourly.with_columns(
    (pl.col("seen_before_slot") / pl.col("total_txs") * 100).alias("coverage_pct"),
)

df_hourly_pd = df_hourly.to_pandas()

fig = px.line(
    df_hourly_pd,
    x="hour",
    y="coverage_pct",
    color="tx_type_label",
    color_discrete_map={v: TX_TYPE_COLORS[k] for k, v in TX_TYPE_LABELS.items()},
    labels={"hour": "Time", "coverage_pct": "Seen before slot (%)", "tx_type_label": "Type"},
    markers=True,
)
fig.update_layout(
    margin=dict(l=60, r=30, t=30, b=60),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
    height=400,
)
fig.show(config={"responsive": True})

## Transaction volume over time

Hourly transaction counts split by public (seen in mempool) vs private (not seen). The private portion represents MEV bundles and other transactions submitted directly to builders.

In [ ]:
# Aggregate across types by hour - 3-way breakdown
df_volume = df.group_by("hour").agg(
    pl.col("total_txs").sum(),
    pl.col("seen_before_slot").sum(),
    pl.col("seen_after_slot").sum(),
)
df_volume = df_volume.with_columns(
    (pl.col("total_txs") - pl.col("seen_before_slot") - pl.col("seen_after_slot")).alias("never_seen"),
)

df_volume_pd = df_volume.to_pandas()

fig = go.Figure()
fig.add_trace(go.Bar(
    x=df_volume_pd["hour"],
    y=df_volume_pd["seen_before_slot"],
    name="Before slot (public)",
    marker_color="#27ae60",
))
fig.add_trace(go.Bar(
    x=df_volume_pd["hour"],
    y=df_volume_pd["seen_after_slot"],
    name="After slot (propagated)",
    marker_color="#3498db",
))
fig.add_trace(go.Bar(
    x=df_volume_pd["hour"],
    y=df_volume_pd["never_seen"],
    name="Never seen (private)",
    marker_color="#95a5a6",
))
fig.update_layout(
    barmode="stack",
    margin=dict(l=60, r=30, t=30, b=60),
    xaxis=dict(title="Time"),
    yaxis=dict(title="Transaction count"),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
    height=400,
)
fig.show(config={"responsive": True})

## Coverage heatmap

Heatmap showing mempool visibility over time for each transaction type. Darker colors indicate higher coverage (more transactions seen in the public mempool).

In [ ]:
# Pivot for heatmap using hourly aggregated data
df_pivot = df_hourly.pivot(
    on="hour",
    index="tx_type_label",
    values="coverage_pct",
).fill_null(0)

# Get column order (all columns except tx_type_label, sorted)
value_cols = [c for c in df_pivot.columns if c != "tx_type_label"]
value_cols_sorted = sorted(value_cols)

# Extract data for heatmap
z_values = df_pivot.select(value_cols_sorted).to_numpy()
y_labels = df_pivot["tx_type_label"].to_list()

fig = go.Figure(
    data=go.Heatmap(
        z=z_values,
        x=value_cols_sorted,
        y=y_labels,
        colorscale="Greens",
        colorbar=dict(title=dict(text="Coverage %", side="right")),
    )
)
fig.update_layout(
    margin=dict(l=100, r=30, t=30, b=60),
    xaxis=dict(title="Time"),
    yaxis=dict(title="Transaction type"),
    height=300,
)
fig.show(config={"responsive": True})

## Mempool age distribution

How long transactions waited in the mempool before being included in a block. The age is measured from first observation in our sentries to the slot start time. Only transactions seen *before* their inclusion slot are counted.

In [ ]:
# Extract percentiles by tx_type using polars aggregation
# Filter to rows with valid data, then compute mean percentiles per type
df_with_pcts = df.filter(
    (pl.col("seen_before_slot") > 0) & 
    pl.col("age_percentiles_ms").is_not_null() &
    (pl.col("age_percentiles_ms").list.len() >= 7)
)

# Extract individual percentiles and compute mean per tx_type
df_age = df_with_pcts.group_by(["tx_type", "tx_type_label"]).agg(
    (pl.col("age_percentiles_ms").list.get(0).mean() / 1000).alias("p50"),
    (pl.col("age_percentiles_ms").list.get(1).mean() / 1000).alias("p75"),
    (pl.col("age_percentiles_ms").list.get(2).mean() / 1000).alias("p80"),
    (pl.col("age_percentiles_ms").list.get(3).mean() / 1000).alias("p85"),
    (pl.col("age_percentiles_ms").list.get(4).mean() / 1000).alias("p90"),
    (pl.col("age_percentiles_ms").list.get(5).mean() / 1000).alias("p95"),
    (pl.col("age_percentiles_ms").list.get(6).mean() / 1000).alias("p99"),
)

# Display age table
age_display = df_age.select(
    pl.col("tx_type_label").alias("Type"),
    pl.col("p50").round(1).alias("p50 (s)"),
    pl.col("p75").round(1).alias("p75 (s)"),
    pl.col("p90").round(1).alias("p90 (s)"),
    pl.col("p95").round(1).alias("p95 (s)"),
    pl.col("p99").round(1).alias("p99 (s)"),
)
age_display.to_pandas()

In [ ]:
# Visualize age percentiles as line chart
df_age_long = df_age.unpivot(
    index=["tx_type", "tx_type_label"],
    on=["p50", "p75", "p80", "p85", "p90", "p95", "p99"],
    variable_name="percentile",
    value_name="age_s",
)
# Convert percentile labels to numeric for x-axis
df_age_long = df_age_long.with_columns(
    pl.col("percentile").str.replace("p", "").cast(pl.Int64).alias("pct_num"),
)

df_age_long_pd = df_age_long.to_pandas()

fig = px.line(
    df_age_long_pd,
    x='pct_num',
    y='age_s',
    color='tx_type_label',
    color_discrete_map={v: TX_TYPE_COLORS[k] for k, v in TX_TYPE_LABELS.items()},
    markers=True,
    log_y=True,
    labels={'pct_num': 'Percentile', 'age_s': 'Age (seconds)', 'tx_type_label': 'Type'},
)
fig.update_layout(
    margin=dict(l=60, r=30, t=30, b=60),
    xaxis=dict(tickvals=[50, 75, 80, 85, 90, 95, 99], ticktext=['p50', 'p75', 'p80', 'p85', 'p90', 'p95', 'p99']),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
    height=400,
)
fig.show(config={"responsive": True})

In [ ]:
# Aggregate histogram buckets across all slots per tx type
hist_cols = [f'age_hist_{i}' for i in range(15)]
df_hist = df.group_by(["tx_type", "tx_type_label"]).agg(
    [pl.col(c).sum() for c in hist_cols]
)

# Melt to long format for plotting
df_hist_long = df_hist.unpivot(
    index=["tx_type", "tx_type_label"],
    on=hist_cols,
    variable_name="bucket",
    value_name="count",
)
df_hist_long = df_hist_long.with_columns(
    pl.col("bucket").str.extract(r"(\d+)").cast(pl.Int64).alias("bucket_idx"),
)
df_hist_long = df_hist_long.with_columns(
    pl.col("bucket_idx").replace_strict(dict(enumerate(HIST_LABELS)), default=None).alias("bucket_label"),
)

# Sort by bucket index for proper ordering
df_hist_long = df_hist_long.sort(["tx_type", "bucket_idx"])

df_hist_long_pd = df_hist_long.to_pandas()

fig = px.bar(
    df_hist_long_pd,
    x='bucket_label',
    y='count',
    color='tx_type_label',
    color_discrete_map={v: TX_TYPE_COLORS[k] for k, v in TX_TYPE_LABELS.items()},
    facet_col='tx_type_label',
    facet_col_wrap=2,
    labels={'bucket_label': 'Age bucket', 'count': 'Count', 'tx_type_label': 'Type'},
    category_orders={'bucket_label': HIST_LABELS},
)
fig.update_yaxes(matches=None, showticklabels=True)
fig.update_layout(
    margin=dict(l=60, r=30, t=60, b=100),
    showlegend=False,
    height=600,
)
fig.update_xaxes(tickangle=45)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show(config={"responsive": True})

## Propagation delay (seen after slot)

For transactions first seen in the mempool *after* block inclusion, this measures how long after the slot start they appeared.

In [ ]:
# Extract delay percentiles for transactions seen AFTER slot start
df_with_delay = df.filter(
    (pl.col("seen_after_slot") > 0) & 
    pl.col("delay_percentiles_ms").is_not_null() &
    (pl.col("delay_percentiles_ms").list.len() >= 7)
)

df_delay = df_with_delay.group_by(["tx_type", "tx_type_label"]).agg(
    (pl.col("delay_percentiles_ms").list.get(0).mean() / 1000).alias("p50"),
    (pl.col("delay_percentiles_ms").list.get(1).mean() / 1000).alias("p75"),
    (pl.col("delay_percentiles_ms").list.get(2).mean() / 1000).alias("p80"),
    (pl.col("delay_percentiles_ms").list.get(3).mean() / 1000).alias("p85"),
    (pl.col("delay_percentiles_ms").list.get(4).mean() / 1000).alias("p90"),
    (pl.col("delay_percentiles_ms").list.get(5).mean() / 1000).alias("p95"),
    (pl.col("delay_percentiles_ms").list.get(6).mean() / 1000).alias("p99"),
)

# Display delay table
delay_display = df_delay.select(
    pl.col("tx_type_label").alias("Type"),
    pl.col("p50").round(2).alias("p50 (s)"),
    pl.col("p75").round(2).alias("p75 (s)"),
    pl.col("p90").round(2).alias("p90 (s)"),
    pl.col("p95").round(2).alias("p95 (s)"),
    pl.col("p99").round(2).alias("p99 (s)"),
)
delay_display.to_pandas()

In [ ]:
# Visualize delay percentiles as line chart
df_delay_long = df_delay.unpivot(
    index=["tx_type", "tx_type_label"],
    on=["p50", "p75", "p80", "p85", "p90", "p95", "p99"],
    variable_name="percentile",
    value_name="delay_s",
)
# Convert percentile labels to numeric for x-axis
df_delay_long = df_delay_long.with_columns(
    pl.col("percentile").str.replace("p", "").cast(pl.Int64).alias("pct_num"),
)

df_delay_long_pd = df_delay_long.to_pandas()

fig = px.line(
    df_delay_long_pd,
    x='pct_num',
    y='delay_s',
    color='tx_type_label',
    color_discrete_map={v: TX_TYPE_COLORS[k] for k, v in TX_TYPE_LABELS.items()},
    markers=True,
    log_y=True,
    labels={'pct_num': 'Percentile', 'delay_s': 'Delay (seconds)', 'tx_type_label': 'Type'},
)
fig.update_layout(
    margin=dict(l=60, r=30, t=30, b=60),
    xaxis=dict(tickvals=[50, 75, 80, 85, 90, 95, 99], ticktext=['p50', 'p75', 'p80', 'p85', 'p90', 'p95', 'p99']),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
    height=400,
)
fig.show(config={"responsive": True})

In [ ]:
# Aggregate delay histogram buckets across all slots per tx type
delay_hist_cols = [f'delay_hist_{i}' for i in range(15)]
df_delay_hist = df.group_by(["tx_type", "tx_type_label"]).agg(
    [pl.col(c).sum() for c in delay_hist_cols]
)

# Melt to long format for plotting
df_delay_hist_long = df_delay_hist.unpivot(
    index=["tx_type", "tx_type_label"],
    on=delay_hist_cols,
    variable_name="bucket",
    value_name="count",
)
df_delay_hist_long = df_delay_hist_long.with_columns(
    pl.col("bucket").str.extract(r"(\d+)").cast(pl.Int64).alias("bucket_idx"),
)
df_delay_hist_long = df_delay_hist_long.with_columns(
    pl.col("bucket_idx").replace_strict(dict(enumerate(HIST_LABELS)), default=None).alias("bucket_label"),
)

# Sort by bucket index for proper ordering
df_delay_hist_long = df_delay_hist_long.sort(["tx_type", "bucket_idx"])

df_delay_hist_long_pd = df_delay_hist_long.to_pandas()

fig = px.bar(
    df_delay_hist_long_pd,
    x='bucket_label',
    y='count',
    color='tx_type_label',
    color_discrete_map={v: TX_TYPE_COLORS[k] for k, v in TX_TYPE_LABELS.items()},
    facet_col='tx_type_label',
    facet_col_wrap=2,
    labels={'bucket_label': 'Delay bucket', 'count': 'Count', 'tx_type_label': 'Type'},
    category_orders={'bucket_label': HIST_LABELS},
)
fig.update_yaxes(matches=None, showticklabels=True)
fig.update_layout(
    margin=dict(l=60, r=30, t=60, b=100),
    showlegend=False,
    height=600,
)
fig.update_xaxes(tickangle=45)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show(config={"responsive": True})

## Sentry coverage

How much of the canonical transaction set each mempool observer (sentry) captured. Higher coverage indicates better mempool visibility from that observation point.

In [ ]:
display_sql("sentry_coverage", target_date)

In [ ]:
df_sentry = pl.from_pandas(load_parquet("sentry_coverage", target_date))

# Shorten sentry names for display
df_sentry = df_sentry.with_columns(
    pl.col("sentry").str.replace("ethpandaops/mainnet/", "").alias("sentry_short"),
)

df_sentry_pd = df_sentry.head(15).to_pandas()

fig = px.bar(
    df_sentry_pd,
    x="coverage_pct",
    y="sentry_short",
    orientation="h",
    labels={"coverage_pct": "Coverage (%)", "sentry_short": "Sentry"},
    text="coverage_pct",
)
fig.update_traces(texttemplate="%{text:.1f}%", textposition="outside")
fig.update_layout(
    margin=dict(l=250, r=60, t=30, b=60),
    xaxis=dict(range=[0, 105]),
    yaxis=dict(autorange="reversed"),
    height=500,
)
fig.show(config={"responsive": True})